# Imports

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
import pandas as pd
from datetime import datetime
import numpy as np




In [133]:
df_meteo = pd.read_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/open-meteo-51.49N0.16W23m.csv', sep=';')

df_incidents = pd.read_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/jan-dec-2022-gla-data-extract-attendant.csv')

In [ ]:
df_meteo.columns

In [ ]:
df_meteo.head(100)

In [ ]:
len(df_incidents)

In [ ]:
df_incidents.columns

In [ ]:
df_incidents['Place Collision Reported'].unique()

In [ ]:
df_incidents[['Day Name', 'Time', 'Highway Authority', 'First Road Class', 'First Road Number',
       'Road Type', 'Speed Limit', 'Junction Detail']].head(1)

In [ ]:
df_incidents.head(1)

# Date and time adjusts

## Weather dataset

In [134]:
df_meteo['datetime'] = pd.to_datetime(df_meteo['time'])

df_meteo.drop(['time'], axis='columns', inplace=True)

In [ ]:
df_meteo.head(100)

## Incident dataset

In [135]:
df_incidents['_Collision Date'] = pd.to_datetime(df_incidents['_Collision Date'])
df_incidents['Time'] = pd.to_datetime(df_incidents['Time']).dt.time


In [136]:
# Criando uma nova coluna combinando data e hora
df_incidents['DateTime Rounded Hour'] = df_incidents['_Collision Date'].astype(str) + ' ' + df_incidents['Time'].apply(lambda x: x.hour).astype(str) + ':00:00'

# Convertendo a coluna para o formato desejado
df_incidents['DateTime Rounded Hour'] = pd.to_datetime(df_incidents['DateTime Rounded Hour'], format='%Y-%m-%d %H:%M:%S')


In [137]:
df_incidents.drop(['_Collision Date'], axis='columns', inplace=True)
df_incidents.drop(['Time'], axis='columns', inplace=True)

In [ ]:
df_incidents.columns

# Merge datasets

In [138]:
merged_df = pd.merge(df_incidents, df_meteo, how='left', left_on='DateTime Rounded Hour', right_on='datetime')

merged_df.drop(['datetime'], axis='columns', inplace=True)

merged_df.sort_values('DateTime Rounded Hour', inplace=True)

# Column clear

In [139]:
merged_df.drop(['_Collision Id'], axis='columns', inplace=True)
merged_df.drop(['Borough Name'], axis='columns', inplace=True)
merged_df.drop(['Borough Number'], axis='columns', inplace=True)
merged_df.drop(['Easting'], axis='columns', inplace=True)
merged_df.drop(['Northing'], axis='columns', inplace=True)
merged_df.drop(['Collision Location'], axis='columns', inplace=True)
merged_df.drop(['_Casualty Count'], axis='columns', inplace=True)
merged_df.drop(['_Vehicle Count'], axis='columns', inplace=True)
merged_df.drop(['Highway Authority'], axis='columns', inplace=True)
merged_df.drop(['First Road Class'], axis='columns', inplace=True)
merged_df.drop(['First Road Number'], axis='columns', inplace=True)
merged_df.drop(['Road Type'], axis='columns', inplace=True)
merged_df.drop(['Junction Detail'], axis='columns', inplace=True)
merged_df.drop(['Junction Control'], axis='columns', inplace=True)
merged_df.drop(['Second Road Class'], axis='columns', inplace=True)
merged_df.drop(['Second Road Number'], axis='columns', inplace=True)
merged_df.drop(['Pedestrian Crossing Facilities'], axis='columns', inplace=True)
merged_df.drop(['Weather Details'], axis='columns', inplace=True)
merged_df.drop(['Road Surface Condition'], axis='columns', inplace=True)
merged_df.drop(['Special Conditions at Site'], axis='columns', inplace=True)
merged_df.drop(['Carriageway Hazards'], axis='columns', inplace=True)
merged_df.drop(['Place Collision Reported'], axis='columns', inplace=True)
merged_df.drop(['precipitation (mm)'], axis='columns', inplace=True)
merged_df.drop(['relative_humidity_2m (%)'], axis='columns', inplace=True)


# Data normalization

In [140]:
merged_df['_Collision Severity'] = pd.factorize(merged_df['_Collision Severity'])[0] + 1
merged_df['Day Name'] = pd.factorize(merged_df['Day Name'])[0] + 1
merged_df['Light Conditions'] = pd.factorize(merged_df['Light Conditions'])[0] + 1

In [141]:
merged_df['temperature_2m (°C)'] = (merged_df['temperature_2m (°C)'] - merged_df['temperature_2m (°C)'].min()) / (merged_df['temperature_2m (°C)'].max() - merged_df['temperature_2m (°C)'].min())
merged_df['snowfall (cm)'] = (merged_df['snowfall (cm)'] - merged_df['snowfall (cm)'].min()) / (merged_df['snowfall (cm)'].max() - merged_df['snowfall (cm)'].min())
merged_df['Speed Limit'] = (merged_df['Speed Limit'] - merged_df['Speed Limit'].min()) / (merged_df['Speed Limit'].max() - merged_df['Speed Limit'].min())
merged_df['rain (mm)'] = (merged_df['rain (mm)'] - merged_df['rain (mm)'].min()) / (merged_df['rain (mm)'].max() - merged_df['rain (mm)'].min())


# Data check

In [51]:
len(merged_df)

23465

In [71]:
merged_df.head(1)

,_Collision Severity,Day Name,Speed Limit,Light Conditions,temperature_2m (°C),rain (mm),snowfall (cm),Month,Day,Hour
8632,1,1,0.253731,1,0.418014,0.0,0.0,1,1,0


In [46]:
merged_df['_Collision Severity'].value_counts()

1    19645
2     3720
3      100
Name: _Collision Severity, dtype: int64

In [47]:
merged_df['Day Name'].value_counts()

3    3851
7    3662
5    3442
2    3396
1    3364
6    3111
4    2639
Name: Day Name, dtype: int64

In [48]:
merged_df['Light Conditions'].value_counts()

1    15972
2     6840
4      376
3      145
5      132
Name: Light Conditions, dtype: int64

In [142]:
merged_df['rain (mm)'].value_counts()

0.000000    20002
0.006803     1341
0.013605      581
0.020408      347
0.027211      198
0.034014      166
0.040816      110
0.054422       96
0.068027       80
0.061224       76
0.047619       58
0.074830       53
0.108844       39
0.088435       39
0.081633       37
0.095238       32
0.122449       31
0.115646       17
0.129252       16
0.183673       16
0.102041       13
0.136054       11
0.272109       10
0.170068       10
0.190476       10
0.163265        8
0.142857        8
0.149660        7
0.176871        7
0.299320        6
0.217687        5
0.156463        4
0.197279        4
0.265306        4
0.408163        4
0.863946        3
0.251701        3
0.224490        3
0.707483        2
0.510204        2
0.346939        2
1.000000        1
0.326531        1
0.455782        1
0.258503        1
Name: rain (mm), dtype: int64

In [50]:
merged_df['snowfall (cm)'].value_counts()

0.000000    23426
0.333333       30
0.666667        7
1.000000        2
Name: snowfall (cm), dtype: int64

# Export datasets

In [132]:
merged_df.to_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/generated_dataset.csv')